# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
M. Güdel  ->  M. Güdel  |  ['M. Güdel']


Arxiv has 74 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2306.02465


extracting tarball to tmp_2306.02465...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2306.02465/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'conclusion' from 'tmp_2306.02465/conclusion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'preparations' from 'tmp_2306.02465/preparations.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxi

...

Found 79 bibliographic references in tmp_2306.02471/main.bbl.
syntax error in line 53: '=' expected
Retrieving document from  https://arxiv.org/e-print/2306.02472


extracting tarball to tmp_2306.02472...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2306.02472/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'acknowledgements.tex' from 'tmp_2306.02472/acknowledgements.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'authors_mixed_updated.tex' from 'tmp_2306.02472/authors_mixed_updated.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


/tmp/ipykernel_2038/1863104449.py:49: LatexWarning: 2306.02472 did not run properly
expected string or bytes-like object
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.02465-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.02465) | **Overview of the JWST Advanced Deep Extragalactic Survey (JADES)**  |
|| D. J. Eisenstein, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>A. d. Graaff</mark> |
|*Appeared on*| *2023-06-06*|
|*Comments*| *33 pages, submitted to ApJ Supplement. The JADES Collaboration web site is at this https URL, and the initial data release is available at this https URL with a viewer at this http URL*|
|**Abstract**| We present an overview of the James Webb Space Telescope (JWST) Advanced Deep Extragalactic Survey (JADES), an ambitious program of infrared imaging and spectroscopy in the GOODS-S and GOODS-N deep fields, designed to study galaxy evolution from high redshift to cosmic noon. JADES uses about 770 hours of Cycle 1 guaranteed time largely from the Near-Infrared Camera (NIRCam) and Near-Infrared Spectrograph (NIRSpec) instrument teams. In GOODS-S, in and around the Hubble Ultra Deep Field and Chandra Deep Field South, JADES produces a deep imaging region of ~45 arcmin$^2$ with an average of 130 hrs of exposure time spread over 9 NIRCam filters. This is extended at medium depth in GOODS-S and GOODS-N with NIRCam imaging of ~175 arcmin$^2$ with an average exposure time of 20 hrs spread over 8-10 filters. In both fields, we conduct extensive NIRSpec multi-object spectroscopy, including 2 deep pointings of 55 hrs exposure time, 14 medium pointings of ~12 hrs, and 15 shallower pointings of ~4 hrs, targeting over 5000 HST and JWST-detected faint sources with 5 low, medium, and high-resolution dispersers covering 0.6-5.3 microns. Finally, JADES extends redward via coordinated parallels with the JWST Mid-Infrared Instrument (MIRI), featuring ~9 arcmin$^2$ with 43 hours of exposure at 7.7 microns and twice that area with 2-6.5 hours of exposure at 12.8 microns For nearly 30 years, the GOODS-S and GOODS-N fields have been developed as the premier deep fields on the sky; JADES is now providing a compelling start on the JWST legacy in these fields. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.02470-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.02470) | **JADES: Differing assembly histories of galaxies -- Observational  evidence for bursty SFHs and (mini-)quenching in the first billion years of  the Universe**  |
|| T. J. Looser, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2023-06-06*|
|*Comments*| **|
|**Abstract**| We use deep NIRSpec spectroscopic data from the JADES survey to derive the star formation histories (SFHs) of a sample of 200 galaxies at 0.6$<$z$<$11 and spanning stellar masses from $\rm 10^6$ to $\rm 10^{9.5}~M_\odot$. We find that galaxies at high-redshift, galaxies above the Main Sequence (MS) and low-mass galaxies tend to host younger stellar populations than their low-redshift, massive, and below the MS counterparts. Interestingly, the correlation between age, M$_*$ and SFR existed even earlier than Cosmic Noon, out to the earliest cosmic epochs. However, these trends have a large scatter. Indeed, there are examples of young stellar populations also below the MS, indicating recent (bursty) star formation in evolved systems. We explore further the burstiness of the SFHs by using the ratio between SFR averaged over the last 10 Myr and averaged between 10 Myr and 100 Myr before the epoch of observation ($\mathrm{SFR_{cont, 10}/SFR_{cont, 90}}$). We find that high-redshift and low-mass galaxies have particularly bursty SFHs, while more massive and lower-redshift systems evolve more steadily. We also present the discovery of another (mini-)quenched galaxy at z = 4.4 (in addition to the one at z=7.3 reported by Looser et al. 2023), which might be only temporarily quiescent as a consequence of the extremely bursty evolution. Finally, we also find a steady decline of dust reddening of the stellar population approaching the earliest cosmic epochs, although some dust reddening is still observed in some of the highest redshift and most star forming systems. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.02471-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.02471) | **JADES: The emergence and evolution of Ly-alpha emission & constraints on  the IGM neutral fraction**  |
|| G. C. Jones, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2023-06-06*|
|*Comments*| *18 pages, 10 figures. Submitted to A&A*|
|**Abstract**| The rest-frame UV recombination emission line Ly-alpha can be powered by ionizing photons from young massive stars in star forming galaxies, but its ability to be resonantly scattered by neutral gas complicates its interpretation. For reionization era galaxies, a neutral intergalactic medium (IGM) will scatter Ly-alpha from the line of sight, making Ly-alpha a useful probe of the neutral fraction at z>6. Here, we explore Ly-alpha in JWST/NIRSpec spectra from the ongoing GTO program JADES, which targets hundreds of galaxies in the well-studied GOODS-S and GOODS-N fields. These sources are UV-faint (-20.4<MUV<-16.4), and thus represent a poorly-explored class of galaxies. The low spectral resolution (R~100) spectra of a subset of 93 galaxies in GOODS-S with z_spec>5.5 (as derived with optical lines) are fit with line and continuum models, in order to search for significant line emission. Through exploration of the R100 data, we find evidence for Ly-alpha in 15 sources. Additional analysis of the R1000 data from the same set of galaxies results in five additional detections. This sample allows us to place observational constraints on the fraction of galaxies with Ly-alpha emission in the redshift range 5.5<z<7.5, with a decrease from z=6 to z=7. We also find a positive correlation between Ly-alpha equivalent width and MUV, as seen in other samples. These results are used to estimate the neutral gas fraction at z~7, agreeing with previous results (XHI~0.5-0.9). |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.02472-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.02472) | **Inside-out growth in the early Universe: a core in a vigorously  star-forming disc**  |
|| W. M. Baker, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2023-06-06*|
|*Comments*| *49 pages, 15 figures, submitted to Nature Astronomy*|
|**Abstract**| The physical processes that establish the morphological evolution and the structural diversity of galaxies are key unknowns in extragalactic astrophysics. Here we report the finding of the morphologically-mature galaxy JADES-GS+53.18343-27.79097, which existed within the first 700 million years of the Universe's history. This star-forming galaxy with a stellar mass of $10^{8.6}$ solar masses consists of three components, a highly-compact core with a half-light radius of 144 pc, a strongly star-forming disc with a radius of 468 pc, and a star-forming clump, which all show distinctive star-formation histories. The central stellar mass density of this galaxy is within a factor of two of the most massive present-day ellipticals, while being globally 1000 times less massive. The radial profile of the specific star-formation rate is strongly rising toward the outskirts. This evidence strongly suggests the first detection of inside-out growth of a galaxy as a proto-bulge and a star-forming disc in the Epoch of Reionization. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error expected string or bytes-like object</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.02467-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.02467) | **JADES NIRSpec Initial Data Release for the Hubble Ultra Deep Field:  Redshifts and Line Fluxes of Distant Galaxies from the Deepest JWST Cycle 1  NIRSpec Multi-Object Spectroscopy**  |
|| A. J. Bunker, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2023-06-06*|
|*Comments*| *Submitted to A&A. Data products available from this https URL*|
|**Abstract**| We describe the NIRSpec component of the JWST Deep Extragalactic Survey (JADES), and provide deep spectroscopy of 253 sources targeted with the NIRSpec micro-shutter assembly in the Hubble Ultra Deep Field and surrounding GOODS-South. The multi-object spectra presented here are the deepest so far obtained with JWST, amounting to up to 28 hours in the low-dispersion ($R\sim 30-300$) prism, and up to 7 hours in each of the three medium-resolution $R\approx 1000$ gratings and one high-dispersion grating, G395H ($R\approx2700$). Our low-dispersion and medium-dispersion spectra cover the wavelength range $0.6-5.3\mu$m. We describe the selection of the spectroscopic targets, the strategy for the allocation of targets to micro-shutters, and the design of the observations. We present the public release of the reduced 2D and 1D spectra, and a description of the reduction and calibration process. We measure spectroscopic redshifts for 178 of the objects targeted extending up to $z=13.2$. We present a catalog of all emission lines detected at $S/N>5$, and our redshift determinations for the targets. Combined with the first JADES NIRCam data release, these public JADES spectroscopic and imaging datasets provide a new foundation for discoveries of the infrared universe by the worldwide scientific community. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2306.02465.md
    + _build/html/tmp_2306.02465/./jadesgoodss_may2023_all.png
    + _build/html/tmp_2306.02465/./jadesgoodss_may2023_nsdp1_nsp.png
    + _build/html/tmp_2306.02465/./jadesgoodss_may2023_nsdp2_nsm.png
    + _build/html/tmp_2306.02465/./jadesgoodss_may2023_ncdeep_ncmedcy1_ncmedcy2_ncdpcy1_ncdpcy2_ncmp_others.png
    + _build/html/tmp_2306.02465/./jadesgoodss_may2023_midp_mimed.png
    + _build/html/tmp_2306.02465/./line_sensitivity_v2_1.png
    + _build/html/tmp_2306.02465/./jadesgoodsn_may2023_all.png
    + _build/html/tmp_2306.02465/./jadesgoodsn_may2023_nsp.png
    + _build/html/tmp_2306.02465/./jadesgoodsn_may2023_nsm.png
    + _build/html/tmp_2306.02465/./jadesgoodsn_may2023_nc_ncp_others.png
    + _build/html/tmp_2306.02465/./jadesgoodsn_may2023_mi.png
exported in  _build/html/2306.02470.md
    + _build/html/tmp_2306.02470/./Plots/consolidated_ppxf_and_Beagle_output_x_stellar_mass_ppxf_y_SFR_HA_K12_c_mean_age_ppxf_mass_weighted_min_p_values_S

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\lya}{\ensuremath{{\rm Ly}\alpha}}$
$\newcommand{\kms}{\ensuremath{{\rm\;km\;s^{-1}}}}$
$\newcommand{\Mpc}{\ensuremath{{\rm\;Mpc}}}$
$\newcommand{\Myr}{\ensuremath{{\rm\;Myr}}}$
$\newcommand{\Msun}{\ensuremath{{\rm\;M_\odot}}}$
$\newcommand{\yr}{\ensuremath{{\rm\;yr}}}$
$\newcommand{\cm}{\ensuremath{{\rm\;cm}}}$
$\newcommand{\ergscms}{\ensuremath{{\rm\;ergs\;cm^{-2}\;s^{-1}}}}$
$\newcommand{\ergss}{\ensuremath{{\rm\;ergs\;s^{-1}}}}$
$\newcommand{\mic}{\ensuremath{\mu\rm m}}$
$\newcommand{\todo}[1]{{\color{blue} \tt #1}}$
$\newcommand{\tbc}[1]{#1 ({\color{red} \tt TBC})}$
$\newcommand{\tbd}{({\color{red} \tt TBD})}$
$\newcommand{\outline}[1]{{\color{black}\it #1}}$
$\newcommand{\BRC}[1]{{\color{red!55!black} BR: #1}}$
$\newcommand{\CWC}[1]{{\color{purple!55!black} CW: #1}}$
$\newcommand{\RM}[1]{{\color{green!25!black} RM: #1}}$
$\newcommand{\DJE}[1]{{\color{blue!25!black} DE: #1}}$
$\newcommand{\um}{\ensuremath{\mu{\rm m}}}$
$\newcommand{\nod}{\nodata & \nodata}$</div>



<div id="title">

# Overview of the JWST Advanced Deep Extragalactic Survey (JADES)

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2306.02465-b31b1b.svg)](https://arxiv.org/abs/2306.02465)<mark>Appeared on: 2023-06-06</mark> -  _33 pages, submitted to ApJ Supplement. The JADES Collaboration web site is at this https URL, and the initial data release is available at this https URL with a viewer at this http URL_

</div>
<div id="authors">

D. J. Eisenstein, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** We present an overview of the James Webb Space Telescope (JWST) Advanced Deep Extragalactic Survey (JADES), an ambitious program of infrared imaging and spectroscopy in the GOODS-S and GOODS-N deep fields, designed to study galaxy evolution from high redshift to cosmic noon.  JADES uses about 770 hours of Cycle 1 guaranteed time largely from the Near-Infrared Camera (NIRCam) and Near-Infrared Spectrograph (NIRSpec) instrument teams.In GOODS-S, in and around the Hubble Ultra Deep Field and ChandraDeep Field South, JADES produces a deep imaging region of $\sim$ 45arcmin $^2$ with an average of 130 hrs of exposure time spreadover 9 NIRCam filters.  This is extended at medium depth in GOODS-Sand GOODS-N with NIRCam imaging of $\sim$ 175 arcmin $^2$ with an averageexposure time of 20 hrs spread over 8--10 filters.  In bothfields, we conduct extensive NIRSpec multi-object spectroscopy,including 2 deep pointings of 55 hrs exposure time, 14 mediumpointings of $\sim$ 12 hrs, and 15 shallower pointings of $\sim$ 4 hrs, targetingover 5000 HST and JWST-detected faint sources with 5 low, medium, and high-resolution dispersers covering 0.6--5.3 $\mu$ m.  Finally, JADESextends redward via coordinated parallels with the JWST Mid-InfraredInstrument (MIRI), featuring $\sim$ 9 arcmin $^2$ with 43hours of exposure at 7.7 $\mic$ and twice that area with 2--6.5hours of exposure at 12.8 $\mic$ .  For nearly 30 years, the GOODS-Sand GOODS-N fields have been developed as the premier deep fieldson the sky; JADES is now providing a compelling start on the JWSTlegacy in these fields.

</div>

<div id="div_fig1">

<img src="tmp_2306.02465/./jadesgoodss_may2023_all.png" alt="Fig6.1" width="20%"/><img src="tmp_2306.02465/./jadesgoodss_may2023_nsdp1_nsp.png" alt="Fig6.2" width="20%"/><img src="tmp_2306.02465/./jadesgoodss_may2023_nsdp2_nsm.png" alt="Fig6.3" width="20%"/><img src="tmp_2306.02465/./jadesgoodss_may2023_ncdeep_ncmedcy1_ncmedcy2_ncdpcy1_ncdpcy2_ncmp_others.png" alt="Fig6.4" width="20%"/><img src="tmp_2306.02465/./jadesgoodss_may2023_midp_mimed.png" alt="Fig6.5" width="20%"/>

**Figure 6. -** Layout of the JADES observations in the GOODS-South field. JADES observations with NIRCam, NIRSpec MOS, and MIRI are shown as colored shaded regions. Higher opacity indicates higher exposure time for NIRCam and MIRI or overlapping MSA pointings for NIRSpec. Dithers and nods smaller than 2 arcseconds are not plotted. For NIRCam, only the SW quadrants are shown for clarity. For NIRSpec, only the active area of the MSA that was used for target placement, excluding regions that lead to truncated prism spectra, is shown. Some of the observations yet to be made in Cycle 2 (NIRSpec Deep/JWST and most Medium/JWST and their associated NIRCam parallels) have positions and orientations that are still to be defined based on scheduling. Outlines of other surveys, including the HST/ACS UDF and CANDELS, are shown with black and olive green curves. The smaller sub-panels show the same information split by instrument for clarity because it can be difficult to see the details when all observations are plotted together. The NIRCam sub-plot in upper-right additionally includes field outlines for the public JWST Cycle 1 NIRCam imaging from the JEMS, FRESCO and NGDEEEP programs.
 (*fig:south*)

</div>
<div id="div_fig2">

<img src="tmp_2306.02465/./line_sensitivity_v2_1.png" alt="Fig2" width="100%"/>

**Figure 2. -** The unresolved line detection limit as a function of wavelength for two JADES NIRSpec tiers.  We assume a well-centered point source and use a $10\sigma$ detection threshold.
The blue long lines are the prism, while the orange, green, and cyan lines are the three $R=1000$ gratings.  The solid lines show the depth in the Deep/HST pointing, while the dashed lines are a representative Medium/JWST pointing.
For an unresolved line, the G395H grating has a similar line detection limit to G395M plotted here.
 (*fig:ns_depth*)

</div>
<div id="div_fig3">

<img src="tmp_2306.02465/./jadesgoodsn_may2023_all.png" alt="Fig7.1" width="20%"/><img src="tmp_2306.02465/./jadesgoodsn_may2023_nsp.png" alt="Fig7.2" width="20%"/><img src="tmp_2306.02465/./jadesgoodsn_may2023_nsm.png" alt="Fig7.3" width="20%"/><img src="tmp_2306.02465/./jadesgoodsn_may2023_nc_ncp_others.png" alt="Fig7.4" width="20%"/><img src="tmp_2306.02465/./jadesgoodsn_may2023_mi.png" alt="Fig7.5" width="20%"/>

**Figure 7. -** Layout of the JADES observations in the GOODS-North field. Details as in Figure \ref{fig:south}.
GOODS-North data collection is nearly complete, so this figure is nearly final.
Only the southern-most of the Medium/JWST pointings remains, due to a guide-star acquisition failure; this pointing will be repeated at an unknown location and position angle.  The NIRCam sub-plot in the upper-right additionally includes the outlines of the public JWST Cycle 1 NIRCam imaging from the FRESCO program.
 (*fig:north*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2306.02465"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\niii}{N \textsc{iii}]}$
$\newcommand{\niv}{N \textsc{iv}]}$
$\newcommand{\hei}{He \textsc{i}}$
$\newcommand{\heii}{He \textsc{ii}}$
$\newcommand{\ciii}{C \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\oiiiSF}{O \textsc{iii}]}$
$\newcommand{\neiii}{[Ne \textsc{iii}]}$
$\newcommand{\niiiL}{N \textsc{iii}] \lambda}$
$\newcommand{\nivL}{N \textsc{iv}] \lambda}$
$\newcommand{\ciiiL}{C \textsc{iii}] \lambda}$
$\newcommand{\ciiiLL}{C \textsc{iii}] \lambda\lambda}$
$\newcommand{\civ}{C \textsc{iv}}$
$\newcommand{\oiiL}{[O \textsc{ii}] \lambda\lambda}$
$\newcommand{\oiiiL}{[O \textsc{iii}] \lambda}$
$\newcommand{\oiiiSFL}{O \textsc{iii}] \lambda\lambda}$
$\newcommand{\neiiiL}{[Ne \textsc{iii}] \lambda}$
$\newcommand{\hd}{H \delta}$
$\newcommand{\hg}{H \gamma}$
$\newcommand{\hb}{H \beta}$
$\newcommand{\mgii}{Mg \textsc{ii}}$
$\newcommand{\oiiiau}{[O \textsc{iii}]}$
$\newcommand{\oiiiauL}{[O \textsc{iii}] \lambda}$
$\newcommand{\xhi}{x_{\rm \textsc{hi}}}$
$\newcommand{\flux}{erg s^{-1} cm^{-2}}$
$\newcommand{\kms}{km s^{-1}\xspace}$
$\newcommand{\asec}{^{\prime\prime}}$
$\newcommand{\sfr}[2]{\ensuremath{\mathrm{SFR_{#1,#2}}}\xspace}$
$\newcommand{\cont}{{\ensuremath{\mathrm{cont}}}\xspace}$
$\newcommand{\sfrcten}{\sfr{\cont}{10}}$
$\newcommand{\sfrcninety}{\sfr{\cont}{90}}$
$\newcommand{\sfrchund}{\sfr{\cont}{100}}$
$\newcommand{\sfrnebten}{\sfr{neb}{10}}$
$\newcommand{\jadesdeep}{JADES/DEEP-HST}$
$\newcommand{\todo}[1]{\textcolor{orange}{[to do: #1]}}$
$\newcommand{\todiscuss}[1]{\textcolor{teal}{[to discuss: #1]}}$
$\newcommand{\TBC}[1]{\textcolor{violet}{[TBC: #1]}}$
$\newcommand{\txn}[1]{\textnormal{#1}}$
$\newcommand{\gtsim}{\mbox{{\raisebox{-0.4ex}{\stackrel{>}{{\scriptstyle\sim}}$
$}}}}$
$\newcommand{\ltsim}{\mbox{{\raisebox{-0.4ex}{\stackrel{<}{{\scriptstyle\sim}}$
$}}}}$
$\newcommand{\ssim}{\ensuremath{\sim\!}\xspace}$
$\newcommand{\lthan}{\ensuremath{\!<\!}\xspace}$
$\newcommand{\prob}{\hbox{\txn{P}}}$
$\newcommand{\conditional}[2]{\hbox{\txn{P}(#1 \mid #2)}}$
$\newcommand{\range}[3]{\hbox{#1 \sim #2  \txn{--}  #3}}$
$\newcommand{\citationneeded}{\textcolor{ForestGreen}{^{\rm citation\;needed}}}$
$\newcommand{\M}{\hbox{\txn{M}}}$
$\newcommand{\hda}{\ensuremath{\mathrm{H\updelta_A}}\xspace}$
$\newcommand{\re}{\ensuremath{R_\mathrm{e}}\xspace}$
$\newcommand{\Mstar}{\ensuremath{M_\star}\xspace}$
$\newcommand{\tquench}{\ensuremath{\mathrm{t_{quench}}}\xspace}$
$\newcommand{\tform}{\ensuremath{\mathrm{t_{form}}\xspace}}$
$\newcommand{\lambdar}{\ensuremath{\lambda_{\rm R}}\xspace}$
$\newcommand{\eps}{\ensuremath{\epsilon}\xspace}$
$\newcommand{\MSun}{\ensuremath{{\rm M}_\odot}\xspace}$
$\newcommand{\Met}{\ensuremath{\mathrm{log_{10}(Z/Z_{\odot})}}\xspace}$
$\newcommand{\fluxcgs}{\ensuremath{\mathrm{erg s^{-1} cm^{-2}}}\xspace}$
$\newcommand{\targetidfull}{JADES-GS+53.15508-27.80178\xspace}$
$\newcommand{\targetid}{JADES-GS-z7-01-QU\xspace}$
$\newcommand{\fesc}{\ensuremath{f_{\rm esc}}\xspace}$
$\newcommand{\Mgas}{\hbox{\M_\txn{gas}}}$
$\newcommand{\Mbh}{\hbox{M_{\textsc{bh}}}}$
$\newcommand{\Mtot}{\hbox{\M_\txn{tot}}}$
$\newcommand{\MtotInLog}{\hbox{\txn{M}_\txn{tot}}}$
$\newcommand{\yr}{\hbox{\txn{yr}}}$
$\newcommand{\LymanAlpha}{\text{Ly\alpha}\xspace}$
$\newcommand{\Halpha}{H\alpha\xspace}$
$\newcommand{\Hbeta}{H\beta\xspace}$
$\newcommand{\Hgamma}{H\gamma\xspace}$
$\newcommand{\DeltaMS}{\Delta_{MS}\xspace}$
$\newcommand{\Hdelta}{\text{H\textdelta}\xspace}$
$\newcommand{\OIII}{\text{[O {\sc iii}]\textlambda5008}\xspace}$
$\newcommand{\SiIV}{\text{[Si {\sc iv}]}\xspace}$
$\newcommand{\NII}{{[N {\sc ii}]}\xspace}$
$\newcommand{\Lsun}{\hbox{ {\rm L}_\odot}}$
$\newcommand{\ergscm}{\hbox{\textrm{erg} \textrm{s}^{-1} \textrm{cm}^{-2}}}$
$\newcommand{\jwst}{\textit{JWST}\xspace}$
$\newcommand{\ppxf}{{\sc ppxf}\xspace}$
$\newcommand{\bagpipes}{{\sc bagpipes}\xspace}$
$\newcommand{\beagle}{{\sc beagle}\xspace}$
$\newcommand{\prospector}{{\sc prospector}\xspace}$
$\newcommand{\forcepho}{{\sc forcepho}\xspace}$
$\newcommand{Å}{\oldAA\xspace}$</div>



<div id="title">

# JADES: Differing assembly histories of galaxies - Observational evidence for bursty SFHs and (mini-)quenching in the first billion years of the Universe

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2306.02470-b31b1b.svg)](https://arxiv.org/abs/2306.02470)<mark>Appeared on: 2023-06-06</mark> - 

</div>
<div id="authors">

T. J. Looser, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** We use deep NIRSpec spectroscopic data from the JADES survey to derive the star formation histories (SFHs) of a sample of 200 galaxies at 0.6 $<$ z $<$ 11 and spanning stellar masses from $\rm 10^6$ to $\rm 10^{9.5} M_\odot$ .We find that galaxies at high-redshift, galaxiesabove the Main Sequence (MS) and low-mass galaxies tend to host younger stellar populations than their low-redshift, massive, and below the MS counterparts. Interestingly,the correlation between age, M $_*$ and SFR existed even earlier than Cosmic Noon, out to the earliest cosmic epochs.However, these trends have a large scatter.Indeed, there are examples of young stellar populations also below the MS, indicating recent (bursty) star formation in evolved systems.We explore further the burstiness of the SFHs by using the ratio between SFR averaged over the last 10 Myr and averaged between 10 Myr and 100 Myr before the epoch of observation ( $\sfrcten$ / $\sfrcninety$ ).We find thathigh-redshift and low-mass galaxies have particularly bursty SFHs, while more massive and lower-redshift systems evolve more steadily.We also present the discovery ofanother (mini-)quenched galaxy at z = 4.4 (in addition to the one at z=7.3 reported by Looser et al. 2023), which might be only temporarily quiescent as a consequence of the extremely bursty evolution. Finally, we also find a steady decline of dust reddening of the stellar population approaching the earliest cosmic epochs, although some dust reddening is still observed in some of the highest redshift and most star forming systems.

</div>

<div id="div_fig1">

<img src="tmp_2306.02470/./Plots/consolidated_ppxf_and_Beagle_output_x_stellar_mass_ppxf_y_SFR_HA_K12_c_mean_age_ppxf_mass_weighted_min_p_values_S_N_cut_0.png" alt="Fig5" width="100%"/>

**Figure 5. -** SFR-mass plane color-coded by the average mass-weighted stellar ages measured by $\ppxf$ in three different redshift bins. Each data-point represents a single galaxy. The blue lines represent a simple linear fit of the star-forming main sequence (MS) for this sample at that redshift bin. For reference, the three dotted lines indicate the quenched threshold for the lowest, middle and highest redshifts in each subplot, see main text. The error bar in the top-left corner represent the RMS-errors for $\M$star and SFR for the entire sample. Quiescent and (mini-)quenched galaxies, for which no SFR could be estimated due to non-detection of the relevant nebular emission lines, are plotted at the bottom of the three sub-figures.
 (*fig:Stellar-ages*)

</div>
<div id="div_fig2">

<img src="tmp_2306.02470/./Plots/binned_SFH_MW_1D_0.6_z_2_min_p_value_-0.05.png" alt="Fig6" width="100%"/>

**Figure 6. -** Mass-weighted stacks of "normalized" SFHs of galaxies in three bins of each, $\M$star and $\DeltaMS$, in the redshift range $\mathrm{0<z<2}$. The number in the legend of each sub-figure indicates the number of galaxies contributing to the stack. In each bin, the SSP-weights of each contributing galaxy are first normalized, and then averaged over all galaxies contributing to the bin. The underlying inferred SSP weight-grid  is collapsed into four age-bins, where each bin has a width of 1 dex, as indicated. The oldest bin (from 1 Gyr to 10 Gyr) is artificially extended for consistent plotting: For each individual galaxy SFH, only SSP templates consistent with the age of the Universe at the redshift of the target are included in the fitting. See section $\re$f{ppxf_methodology} for more details. (*fig:SFH_MW_bin_1*)

</div>
<div id="div_fig3">

<img src="tmp_2306.02470/./Plots/consolidated_ppxf_and_Beagle_output_x_stellar_mass_ppxf_y_SFR_HA_K12_c_SFR_ppxf_10Myr_div_90Myr_min_p_values_S_N_cut_0.png" alt="Fig12" width="100%"/>

**Figure 12. -** Observational evidence for bursty SFHs: SFR-mass plane color-coded by the ratio of the SFR over the last 10 Myr ($\sfr$cten), and 10 Myr to 100 Myr ($\sfr$cninety) before the epoch of observation. Both tracers are inferred from non-parametric SSP fitting of the stellar continuum with $\ppxf$. Each data-point represents a single galaxy. The quiescent galaxies are plotted at the bottom of the subplots. More details are given in Fig. $\re$f{fig:Stellar-ages}. (*SFR_mass_plane_burstiness*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2306.02470"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\cii}{[C II]\xspace}$
$\newcommand{\oiii}{[O III]\xspace}$
$\newcommand{\lya}{Ly\alpha\xspace}$
$\newcommand{\angstrom}{\textup{Å}\xspace}$
$\newcommand{\rew}{REW_{\rm Ly\alpha}}$</div>



<div id="title">

# JADES: The emergence and evolution of Ly$\alpha$ emission \& constraints on the IGM neutral fraction

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2306.02471-b31b1b.svg)](https://arxiv.org/abs/2306.02471)<mark>Appeared on: 2023-06-06</mark> -  _18 pages, 10 figures. Submitted to A&A_

</div>
<div id="authors">

G. C. Jones, et al. -- incl., <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** The rest-frame UV recombination emission line $\lya$ can be powered by ionizing photons from young massive stars in star forming galaxies, but its ability to be resonantly scattered by neutral gas complicates its interpretation. For reionization era galaxies, a neutral intergalactic medium (IGM) will scatter $\lya$ from the line of sight, making $\lya$ a useful probe of the neutral fraction at $z\gtrsim6$ . Here, we explore $\lya$ in JWST/NIRSpec spectra from the ongoing GTO program JADES, which targets hundreds of galaxies in the well-studied GOODS-S and GOODS-N fields. These sources are UV-faint ( $-20.4<\rm M_{\rm UV}<-16.4$ ), and thus represent a poorly-explored class of galaxies. The low spectral resolution ( $R\sim100$ ) spectra of a subset of 93 galaxies in GOODS-S with $z_{spec}>5.5$ (as derived with optical lines) are fit with line and continuum models, in order to search for significant line emission. Through exploration of the R100 data, we find evidence for $\lya$ in 15 sources. Additional analysis of the R1000 data from the same set of galaxies results in five additional detections. This sample allows us to place observational constraints on the fraction of galaxies with $\lya$ emission in the redshift range $5.5<z<7.5$ , with a decrease from $z=6$ to $z=7$ . We also find a positive correlation between $\lya$ equivalent width and M $_{UV}$ , as seen in other samples. These results are used to estimate the neutral gas fraction at $z\sim7$ , agreeing with previous results ( $X_{HI}\sim0.5-0.9$ ).

</div>

<div id="div_fig1">

<img src="tmp_2306.02471/./MUV_z_May31_V1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Systemic redshift (based on optical lines) versus M$_{\rm UV}$(from NIRSpec spectra, see Appendix \ref{m1500app}) for our sample. Galaxies observed in different tiers are coloured differently. Sources detected in $\lya$ emission have black outlines. Vertical dashed lines show M$_{\rm UV}$ values of -18.75 and -20.25. (*fig_zmuv*)

</div>
<div id="div_fig2">

<img src="tmp_2306.02471/./Damping_V4.png" alt="Fig3" width="100%"/>

**Figure 3. -** Transmission models of a damping wing (brown lines) and a step function (green lines) for a $z=10$ source. We include the intrinsic model, regridded to match our R100 observations (solid lines), and the dispersed version of this model (dashed lines). The $\lya$ wavelength is shown by a faint vertical line. (*damp*)

</div>
<div id="div_fig3">

<img src="tmp_2306.02471/./XHI_May31_V3.png" alt="Fig2" width="100%"/>

**Figure 2. -** Cumulative distribution for $\lya$ rest EW for UV-faint ($-20.25<M_{UV}<-18.75$) galaxies at $z\sim7$. Each solid line shows the expected distribution for a model with $N_{HI}=10^{20}$ cm$^{-2}$ and a wind speed of 200 km s$^{-1}$, but with a different neutral fraction \citep{pent14}. Estimates from literature are shifted by $1$\angstrom$$ for visibility. We include both the estimated distribution from our sample and the associated upper limit (see Section \ref{lyafracsec}) (*xhiplot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2306.02471"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

216  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
